In [ ]:
def open_NEON_presence_cover_plant(site,date):
    '''Uses API call to retrieve NEON product data
    for vegetation presence and coverage at a given 
    site and date. Returns pandas of 1 meter squared
    and 10&100 meter squared documents. For more 
    information on NEON presence cover plant data
    see https://data.neonscience.org/data-products/DP1.10058.001
    Parameters
    ----------
    site : str
        4 Letter site name. See 
        https://www.neonscience.org/field-sites/field-sites-map/list
        for a full list of NEON sites
    date : str
        Date of data collection in yyyy-mm format
    Returns
    -------
    small_subplot : pandas
        Pandas of the 1x1 meter subplots
    large_subplot : pandas 
        Pandas of the 10x10 and 100x100 meter
        subplots
    '''
    data_product_url=['https://data.neonscience.org/api/v0/data/DP1.10058.001/'
                      +site+'/'+date+'?package=expanded']
    call_response = requests.get(data_product_url[0])
    data_locations=[]
    for i in call_response.json()['data']['files']:
        data_file_url=i['url']
        file_format=data_file_url.find('10m2Data')
        if not file_format == -1:
            data_locations.append(data_file_url)
   # small_subplot = pd.read_csv(data_locations[1]).dropna(axis=1)
    large_subplot = pd.read_csv(data_locations[0]).dropna(axis=1)
    return large_subplot